# Simple MNIST MLP Classifier with regularization and Hyperparameter search

**Description:** Simple example for Multi Layer Perceptron MNIST Classifier included several regularization 
examples like dropout, learning rate, early stopping<br>
This example has larger layers and more complex training<br>
**Dataset:** Classical MNIST dataset <br>
.

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # to avoid warning messages

import tensorflow as tf
import matplotlib.pyplot as plt

import keras_tuner as kt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Model


def Analyze_training(h):
# summarize history for accuracy
# history.history.keys() -> dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
   plt.plot(h.history['accuracy'])
   plt.plot(h.history['val_accuracy'])
   plt.title('model accuracy')
   plt.ylabel('accuracy')
   plt.xlabel('epoch')
   plt.legend(['train', 'test'], loc='upper left')
   plt.show()
# summarize history for loss
   plt.plot(h.history['loss'])
   plt.plot(h.history['val_loss'])
   plt.title('model loss')
   plt.ylabel('loss')
   plt.xlabel('epoch')
   plt.legend(['train', 'test'], loc='upper left')
   plt.show()
   return

In [2]:
# Data definitions
batch_size = 256
epochs = 10


### Data Preparation

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
y_train.shape

(60000, 10)

### Hyperparameter Search

In [5]:
# MLP definition & Hyperparameter size

def build_model(hp):
    
   ### Optimization of layer sizes
   hlayer1 = hp.Choice('layer1', [8,16,32,128,256,512]) 
   hlayer2 = hp.Int('layer2', min_value=32, max_value=512, step=32)
   hlayer3 = hp.Choice('layer3', [8,16,32,128,256,512]) 

   # iput layer
   inputs = layers.Input(shape=(784,))
   # first hidden layer
   x = layers.Dense(hlayer1, activation='relu')(inputs)
   x = layers.Dropout(0.1)(x)
   # second hidden layer
   x = layers.Dense(hlayer2, activation='relu')(x)
   x = layers.Dropout(0.1)(x)
   # third hidden layer
   x = layers.Dense(hlayer3, activation='relu')(x)
   x = layers.Dropout(0.1)(x)
   #Final Layer (Output)
   output = layers.Dense(10, activation='softmax')(x)
    
   model = Model(inputs=[inputs], outputs=output)
   model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),
              metrics=['accuracy'])   
    
   return model

In [6]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=100,
    overwrite = True)

In [7]:
tuner.search(x_train, y_train, epochs=15, validation_data=(x_test, y_test))
best_model = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 100 Complete [00h 00m 25s]
val_accuracy: 0.9197999835014343

Best val_accuracy So Far: 0.9801999926567078
Total elapsed time: 01h 06m 58s
INFO:tensorflow:Oracle triggered exit


In [9]:
print('best model has layer1:', best_model.get('layer1'), 'layer 2:', best_model.get('layer2'), 'layer 3:', 
       best_model.get('layer3'))

best model has layer1: 256 layer 2: 384 layer 3: 32
